#Enunciados dos Trabalhos para Casa (TPC)
da UC de **Processamento de Linguagens**,
ano Letivo 2022/23 , 3º ano de LEI

Relativos à 1º parte do programa: Filtros de Texto baseados em ER
**texto em negrito**

#TPC3: Processador de Pessoas listadas nos Róis de Confessados

(publicado em **2023.02.28**)

Construa agora um ou vários programas Python
para  processar o texto 'processos.txt' (procurar o ficheiro no Bb) com o intuito de
calcular frequências de alguns elementos (a ideia é utilizar arrays associativos, dicionários em Python,
para o efeito) conforme solicitado a seguir:

> a) Calcula a frequência de processos por ano (primeiro elemento da data);

> b) Calcula a frequência de nomes próprios (o primeiro em cada nome) e apelidos (o ultimo em cada nome) por séculos e apresenta os 5 mais usados;

> c) Calcula a frequência dos vários tipos de relação: irmão, sobrinho, etc.;

> d) Converta os 20 primeiros registos num novo ficheiro de output mas em formato **Json**.




Processamento do ficheiro 

In [3]:
import re
import json
from prettytable import PrettyTable

data = []
pattern = re.compile(r'^(?P<folder>\d+)::(?P<date>\d{4}-\d{2}-\d{2})::(?P<name>((?i)[a-z]+\s)+(?i)[a-z]+)::(?P<father>((?i)[a-z]+\s)+(?i)[a-z]+)::(?P<mother>((?i)[a-z]+\s)+(?i)[a-z]+)::(?P<obs>.*)::$')
pattern_obs = re.compile(r'(([A-Z][a-z]+\s?)+),(([A-Z][a-z]+\s?)+)')
with open('processos.txt', 'r') as f:
    for line in f:
        if line.strip() != '':
            if re.match(pattern, line):
                person = pattern.search(line).groupdict()
                obs = []
                if person['obs'] != '' and re.match(pattern_obs, person['obs']):
                    for name, _, relation, _ in re.findall(pattern_obs, person['obs']):
                        obs.append((name, relation))
                person['obs'] = obs
                data.append(person)

a) Calcula a frequência de processos por ano (primeiro elemento da data);

In [4]:
    years = {}
    total = 0
    pattern = re.compile(r'(\d+)')
    myTable = PrettyTable(["Year", "%"])
    for person in data:
        total += 1
        y = pattern.match(person['date']).groups(1)[0]
        if y not in years:
            years[y] = 0
        years[y] += 1
    for key, value in years.items():
        years[key] = (value / total) * 100
        row = []
        row.append(key)
        row.append("{:.4f}".format(years[key]))
        myTable.add_row(row)


    #Table
    for year,freq in sorted(years.items(), key=lambda y: (y[1]), reverse=True):
        row = []
        row.append(year)
        row.append("{:.4f}".format(freq))
        myTable.add_row(row)

    print(myTable)

+------+--------+
| Year |   %    |
+------+--------+
| 1894 | 0.2010 |
| 1909 | 0.1059 |
| 1867 | 0.1467 |
| 1896 | 0.1874 |
| 1904 | 0.1250 |
| 1901 | 0.1575 |
| 1883 | 0.0924 |
| 1900 | 0.1277 |
| 1902 | 0.2173 |
| 1880 | 0.1657 |
| 1889 | 0.1766 |
| 1908 | 0.1304 |
| 1869 | 0.1032 |
| 1862 | 0.1087 |
| 1906 | 0.1711 |
| 1856 | 0.1929 |
| 1875 | 0.0407 |
| 1892 | 0.1440 |
| 1733 | 3.1184 |
| 1778 | 2.8332 |
| 1691 | 2.7408 |
| 1730 | 3.1238 |
| 1899 | 0.2119 |
| 1898 | 0.2472 |
| 1877 | 0.1277 |
| 1910 | 0.0733 |
| 1881 | 0.1684 |
| 1907 | 0.1222 |
| 1884 | 0.1141 |
| 1879 | 0.1440 |
| 1895 | 0.2064 |
| 1897 | 0.1847 |
| 1707 | 0.2961 |
| 1689 | 1.6760 |
| 1713 | 0.7226 |
| 1824 | 0.6737 |
| 1703 | 0.3721 |
| 1720 | 0.4672 |
| 1890 | 0.1168 |
| 1732 | 5.7831 |
| 1683 | 0.3721 |
| 1863 | 0.0788 |
| 1729 | 0.1222 |
| 1694 | 0.1222 |
| 1754 | 0.8231 |
| 1690 | 0.6492 |
| 1755 | 0.8068 |
| 1823 | 0.5623 |
| 1708 | 0.3939 |
| 1757 | 0.0761 |
| 1699 | 0.2336 |
| 1759 | 0.2418 |
| 1712 | 0

b) Calcula a frequência de nomes próprios (o primeiro em cada nome) e apelidos (o ultimo em cada nome) por séculos e apresenta os 5 mais usados;

In [5]:
    centuries = {}
    pattern = re.compile(r'(\d+)')
    for person in data:
        year = int(pattern.match(person['date']).groups(1)[0])
        century = ""
        if (year % 100 == 0):
            century = str(year // 100)
        else:
            century = str((year // 100) + 1)
        if century not in centuries:
            centuries[century] = {"names": {}, "surnames": {}}

        names = [person['name'], person['father'], person['mother']]
        for p in person['obs']:
            names.append(p[0])

        for n in names:
            first = n.split(" ")[0]
            last = n.split(" ")[-1]
            if first not in centuries[century]['names']:
                centuries[century]['names'][first] = 0
            centuries[century]['names'][first] += 1
            if last not in centuries[century]['surnames']:
                centuries[century]['surnames'][last] = 0
            centuries[century]['surnames'][last] += 1

    for key,value in centuries.items():
        allnames = value['names'].items() #todos os nomes do século
        allsurnames = value['surnames'].items() #todos os apelidos do século
        totalnames = sum(list(value['names'].values()))
        totalsurnames = sum(list(value['surnames'].values()))
        for kname, vname in allnames:
            value['names'][kname] = (vname/totalnames) * 100
        for ksurname, vsurname in allsurnames:
            value['surnames'][ksurname] = (vsurname/totalsurnames) * 100
        centuries[key]['names'] = sorted(centuries[key]['names'].items(), key=lambda y: (y[1]), reverse=True)[:5]
        centuries[key]['surnames'] = sorted(centuries[key]['surnames'].items(), key=lambda y: (y[1]), reverse=True)[:5]


    #Table
    for key,value in centuries.items():
        print(f"{key}th Century")
        myTableNames = PrettyTable(["Name", "%"])
        myTableSurnames = PrettyTable(["Surname", "%"])
        for name, freqname in value['names']:
            row = []
            row.append(name)
            row.append("{:.4f}".format(freqname))
            myTableNames.add_row(row)
        for surname, freqsurname in value['surnames']:
            row = []
            row.append(surname)
            row.append("{:.4f}".format(freqsurname))
            myTableSurnames.add_row(row)
        print(myTableNames)
        print(myTableSurnames)


19th Century
+---------+---------+
|   Name  |    %    |
+---------+---------+
|   Jose  | 12.9638 |
| Antonio | 12.3320 |
|  Manuel | 11.3326 |
|  Maria  |  9.1124 |
|   Joao  |  8.9765 |
+---------+---------+
+---------+--------+
| Surname |   %    |
+---------+--------+
|  Silva  | 4.1107 |
| Pereira | 3.8237 |
|  Costa  | 3.0861 |
|  Sousa  | 2.6431 |
|  Araujo | 2.4216 |
+---------+--------+
20th Century
+---------+---------+
|   Name  |    %    |
+---------+---------+
| Antonio | 13.6063 |
|  Manuel | 11.1625 |
|   Jose  | 11.0964 |
|  Maria  | 10.1057 |
|   Joao  |  6.8692 |
+---------+---------+
+-----------+--------+
|  Surname  |   %    |
+-----------+--------+
|   Silva   | 5.5482 |
|   Costa   | 3.5667 |
|  Pereira  | 2.9062 |
| Fernandes | 2.7741 |
| Goncalves | 2.5760 |
+-----------+--------+
18th Century
+---------+---------+
|   Name  |    %    |
+---------+---------+
|  Manuel | 11.6181 |
|   Joao  |  9.7337 |
| Antonio |  9.3426 |
|  Maria  |  8.2405 |
|   Jose  |  7.

c) Calcula a frequência dos vários tipos de relação: irmão, sobrinho, etc.;

In [6]:
    relations = {}
    total = 0
    myTable = PrettyTable(["Relation", "%"])
    for person in data:
        for relation in person['obs']:
            if relation[1] not in relations:
                relations[relation[1]] = 0
            relations[relation[1]] +=1
            total +=1


    for key,value in relations.items():
        relations[key] = (value/total) * 100

    sortRelations = sorted(relations.items(), key=lambda y: (y[1]), reverse=True)
    del sortRelations[-1]
    for relation,freq in sortRelations:
        row = []
        row.append(relation)
        row.append("{:.4f}".format(freq))
        myTable.add_row(row)

    print(myTable)


+--------------------------+---------+
|         Relation         |    %    |
+--------------------------+---------+
|          Irmao           | 53.6458 |
|       Tio Materno        | 10.3527 |
|       Tio Paterno        |  9.2531 |
|     Sobrinho Materno     |  7.0260 |
|     Sobrinho Paterno     |  6.7418 |
|          Primo           |  2.7676 |
|           Pai            |  1.3931 |
|      Irmao Paterno       |  1.3325 |
|          Filho           |  1.2580 |
|      Primo Materno       |  1.1275 |
|     Tio Avo Materno      |  0.9505 |
|      Primo Paterno       |  0.8666 |
|     Tio Avo Paterno      |  0.6756 |
|  Sobrinho Neto Materno   |  0.5871 |
|  Sobrinho Neto Paterno   |  0.4053 |
|          Irmaos          |  0.3681 |
|    Sobrinhos Maternos    |  0.2283 |
|       Avo Materno        |  0.2143 |
|       Neto Materno       |  0.1584 |
|      Irmao Materno       |  0.1351 |
|    Sobrinhos Paternos    |  0.1118 |
|          Filhos          |  0.0652 |
|      Tios Maternos     

d) Converta os 20 primeiros registos num novo ficheiro de output mas em formato Json.

In [8]:
    jsonDict = {}
    jsonDict['Process'] = []

    for i in range(20):

        if len(data[i]['obs']) > 0:

            relations = []
            for name, relation in data[i]['obs']:
                relations.append({"name": name, "relation": relation})

            jsonDict['Process'].append({"folder": data[i]['folder'], "date": data[i]['date'], "name": data[i]['name'], "father": data[i]['father'], "mother": data[i]['mother'],
                                        "obs": relations})
        else:
            jsonDict['Process'].append({"folder": data[i]['folder'], "date": data[i]['date'], "name": data[i]['name'],
                                        "father": data[i]['father'], "mother": data[i]['mother'],
                                        "obs": data[i]['obs']})

    with open('first20.json', 'w') as f:
        f.write(json.dumps(jsonDict, indent=4))